<a href="https://colab.research.google.com/github/dernameistegal/airbnb_price/blob/main/data_utils/munich/translate_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Translation of airbnb dataset into english

In [2]:
from google.colab import drive
import pandas as pd
import torch
drive.mount('/content/drive')

Mounted at /content/drive


## Preliminary works
- add comments_en column with values None
- add lang column with values None
- replace na values in comments colum with "no text"

In [4]:
%%capture
!pip install git+https://github.com/neuml/txtai#egg=txtai[pipeline]

In [2]:
import torch
path = "/content/drive/MyDrive/Colab/airbnb/munich/reviews_workfile.pickle"
reviews = pd.read_pickle(path)

In [10]:
import torch
path = "/content/drive/MyDrive/Colab/airbnb/munich/listings.pickle"
listings = pd.read_pickle(path)

In [17]:
reviews.listing_id.isin(listings.id).all()

True

In [8]:
from torch.utils.data import DataLoader, Dataset

class Reviews(Dataset):
    def __init__(self, reviews):
        super().__init__()
        self.reviews = reviews

    def __getitem__(self, key):
        return self.reviews["comments"].iloc[key], self.reviews.index[key]

    def __len__(self):
        return len(self.reviews)

In [9]:
dataset = Reviews(reviews)
dataloader = DataLoader(dataset, batch_size=200)

# Detect language and sort by it

In [12]:
# detect language
from txtai.pipeline import Translation
translate = Translation(batch=200)
with torch.no_grad():
    for i, (text, id) in enumerate(dataloader):
        text = list(text)
        try: 
            if reviews.loc[id, "lang"].isnull().values.any():
                language = translate.detect(list(text))
                reviews.loc[id, "lang"] = language
        except:
          continue
        last_id = id[-1]
        print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81


In [22]:
reviews = reviews.sort_values(by="lang")

In [32]:
# remove text garbage	
reviews["comments_en"] = reviews["comments_en"].str.replace("<.*?>", "")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [33]:
reviews["comments_en"].iloc[0]

"My stay at Ralph's place was amazing. Place is great and has everything a Traveler would need. Ralph was a great host and always up to help if needed. Place is spacious and close to public transportation.br/>Raff apartment is wonderful and comfortable. Everything the traveller needs. Ralph was an excellent host."

In [6]:
path = "/content/drive/MyDrive/Colab/airbnb/munich/reviews_workfile.pickle"
import pandas as pd
reviews = pd.read_pickle(path)

In [37]:
reviews["comments"] = reviews["comments"].fillna("no review")

In [40]:
reviews.to_pickle(path)

# Translate text

In [24]:
len(reviews) / 64

298.796875

In [1]:
!python script.py

python3: can't open file 'script.py': [Errno 2] No such file or directory


In [5]:

#@title this is script.py
from txtai.pipeline import Translation
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Create translation model
translate = Translation(batch=45)

path = "/content/drive/MyDrive/Colab/airbnb/munich/reviews_workfile.pickle"
reviews = pd.read_pickle(path)

class Reviews(Dataset):
    def __init__(self, reviews):
        super().__init__()
        self.reviews = reviews

    def __getitem__(self, key):
        return self.reviews["comments"].iloc[key], self.reviews.index[key]

    def __len__(self):
        return len(self.reviews)

dataset = Reviews(reviews)
dataloader = DataLoader(dataset, batch_size=45)


with torch.no_grad():
    for i, (text, id) in enumerate(dataloader):
        text = list(text)
        if reviews.loc[id, "comments_en"].isnull().values.any():

            sourcefinder = reviews.loc[id, "lang"].unique()
            if len(sourcefinder) > 1:
                  print(sourcefinder)
                  sourcefinder = None
            else:
                sourcefinder = sourcefinder[0]
                print(i)

            translation = translate(list(text), "en", source = sourcefinder)
            reviews.loc[id, "comments_en"] = translation
            if i % 1 == 0:
                path = "/content/drive/MyDrive/Colab/airbnb/munich/reviews_workfile.pickle"
                reviews.to_pickle(path)
        else:
            print(i, "already done")



0 already done
1 already done
2 already done
3 already done
4 already done
5 already done
6 already done
7 already done
8 already done
9 already done
10 already done
11 already done
12 already done
13 already done
14 already done
15 already done
16 already done
17 already done
18 already done
19 already done
20 already done
21 already done
22 already done
23 already done
24 already done
25 already done
26 already done
27 already done
28 already done
29 already done
30 already done
31 already done
32 already done
33 already done
34 already done
35 already done
36 already done
37 already done
38 already done
39 already done
40 already done
41 already done
42 already done
43 already done
44 already done
45 already done
46 already done
47 already done
48 already done
49 already done
50 already done
51 already done
52 already done
53 already done
54 already done
55 already done
56 already done
57 already done
58 already done
59 already done
60 already done
61 already done
62 already done
63

In [ ]:
path = "/content/reviews_save"
reviews = pd.read_pickle(path)

In [7]:
path = "/content/drive/MyDrive/Colab/airbnb/munich/translated_reviews.pickle"
reviews.to_pickle(path)